# Carte des arrondissements

Test du chrolopleth plotly avec la gEographie des arrondissements de montrEal

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import json

MAP_FPATH = '../assets/maps/arrondissements_montreal.geojson'

with open(MAP_FPATH) as f:
    arrond = json.load(f)
    
print(arrond.keys())
print(arrond['type'])
print(arrond['crs'])
print(arrond['features'])

id = 'no_qr'
fig = go.Figure(
    go.Choroplethmap(
        geojson=arrond,
        featureidkey='properties.'+id,
        locations=[f['properties'][id] for f in arrond['features']],
        z=[f['properties'][id] for f in arrond['features']],
        hovertext=[f['properties']['nom_qr'] for f in arrond['features']],
        colorscale="Viridis",
        marker_opacity=0.5, marker_line_width=0))
fig.update_layout(map=dict(center=dict(lat=45.5517, lon=-73.7073), style="carto-positron", zoom=8.7))
fig.show()

# Districts Electoraux de MontrEal
Il manque tout ce qui n'est pas techniquement partie de la ville de montrEal, e.g. Westmount

In [ ]:
import plotly.graph_objects as go
import json

MAP_FPATH = '../assets/maps/districts_montreal.geojson'

with open(MAP_FPATH) as f:
    districts = json.load(f)
    
print(districts.keys())
print(districts['type'])
print(districts['name'])
print(len(districts['features']))
print(districts['features'][0]['geometry'])
print(districts['features'][0].keys())
print(districts['features'][0]['properties'].keys())

id = 'id'
fig = go.Figure(go.Choroplethmap(
    geojson=districts,
    featureidkey='properties.'+id,
    locations=[f['properties'][id] for f in districts['features']],
    z=[f['properties'][id] for f in districts['features']],
    hovertext=[f['properties']['nom'] for f in districts['features']],
    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(
    map=dict(
        center=dict(lat=45.5517, lon=-73.7073), zoom=8.7))
fig.show()

# Superposition des arrondissements et circos

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import json

ARROND_FPATH = '../assets/maps/arrondissements_montreal.geojson'
CIRCO_FPATH = '../assets/maps/districts_montreal.geojson'


with open(ARROND_FPATH) as f:
    arrond = json.load(f)
with open(CIRCO_FPATH) as f:
    districts = json.load(f)    

# En bleu les arrondissements
id = 'no_qr'
fig = go.Figure(
    go.Choroplethmap(
        geojson=arrond,
        featureidkey='properties.'+id,
        locations=[f['properties'][id] for f in arrond['features']],
        z=[0]*len(arrond['features']),
        hovertext=[f['properties']['nom_qr'] for f in arrond['features']],
        marker_opacity=0.5, marker_line_width=5, marker_line_color='blue'))

# En rouge les circonscriptions
id = 'id'
fig = fig.add_trace(go.Choroplethmap(
    geojson=districts,
    featureidkey='properties.'+id,
    locations=[f['properties'][id] for f in districts['features']],
    z=[1]*len(districts['features']),
    hovertext=[f['properties']['nom'] for f in districts['features']],
    colorscale="Viridis",
    marker_opacity=0.5, marker_line_width=5, marker_line_color='red'))
fig.update_layout(
    showlegend=True,
    map=dict(
        center=dict(lat=45.5517, lon=-73.7073), zoom=8.7))
fig.show()

# Test des 125 districts Electoraux du QuEbec

In [ ]:
import plotly.graph_objects as go
import numpy as np
import json

MAP_FPATH = '../assets/maps/districts_QC.geojson'

with open(MAP_FPATH) as f:
    districts = json.load(f)
    
districts['features'] = districts['features']
for i in range(len(districts['features'])):
    districts['features'][i]['properties']['ID'] = i

print(districts.keys())
print(districts['type'])
print(districts['name'])
print(districts['crs'])
print(len(districts['features']))
print(districts['features'][0].keys())
for key in districts['features'][0]['properties'].keys():
    print('|\t', key, ':', districts['features'][0]['properties'][key])

id='ID'
fig = go.Figure(go.Choroplethmap(
    geojson=districts,
    featureidkey='properties.'+id,
    locations=[f['properties'][id] for f in districts['features']],
    z=[f['properties'][id] for f in districts['features']],
    hovertext=[f['properties']['NM_CEP'] for f in districts['features']],
    marker_opacity=0.5, marker_line_width=1))

fig.update_geos(
    projection=dict(
        type="conic conformal",
        parallels=[50, 46],
    ))
fig.update_layout(
    map=dict(center=dict(lat=54, lon=-68.5), zoom=3.7),
    width=600, height=800)

fig.show()

## Test du datasets de langues etc

Des donnEes de langue sur l'ensemble du quEbec (enfin les villes principales), juste pour voir comment c'Etait organisE.

In [ ]:
import pandas as pd

FPATH =  '../assets/data/langues.csv'
df = pd.read_csv(FPATH, sep=';')

In [ ]:
# polars est pas mal plus rapide
import polars as pl

FPATH =  '../assets/data/langues.csv'
df = pl.read_csv(FPATH, separator=';')

# Dropping empty columns
df = df.drop(["Symbole"]+[f"Symbole_duplicated_{i}" for i in range(8)])


In [ ]:
print(*df.columns, sep='\n')

In [ ]:
print(df.head())

In [ ]:
print(*df['GEO'].unique().to_list(), sep='\n')

# Test du dataset de données démographiques par circo

In [ ]:
import plotly.graph_objects as go
from unidecode import unidecode
import pandas as pd
import json

MAP_125_FPATH = '../assets/maps/districts_QC.geojson'
DEMOGRAPHICS_FPATH = '../assets/data/donneesSocio2021.csv'

# Get the map
with open(MAP_125_FPATH) as f:
    districts = json.load(f)
print('Number of features :', len(districts['features']))

# Clean the names
for i in range(len(districts['features'])):
    districts['features'][i]['properties']['NM_CEP'] = unidecode(districts['features'][i]['properties']['NM_CEP'])
# Roughly sort. NOT RELIABLE for 1-to-1 matching
districts['features'].sort(key=lambda x: x['properties']['NM_CEP'].lower())
 
# Add unique IDs
for i in range(len(districts['features'])):
    districts['features'][i]['properties']['ID'] = i

# Display the keys
print('Property keys for the map :')
for key in districts['features'][0]['properties'].keys():
    print('|\t', key, ':', type(districts['features'][0]['properties'][key]), '; example :', districts['features'][0]['properties'][key])

# Get the demographics
df = pd.read_csv(DEMOGRAPHICS_FPATH, sep=';')
df.rename(columns={s:unidecode(s) for s in df.columns}, inplace=True)
df.rename(columns={'Circonscription/ DSE 2021':'Circonscription'}, inplace=True)
print('\nDataframe columns :')
print(*df.columns, sep='\n')
df.rename(columns={s: s.strip() for s in df.columns}, inplace=True)

# The first row is a recap for the entire province, we drop it and clean
df = df.drop(df[df['Circonscription']=='Province'].index).reset_index(drop=True)
df['Circonscription'] = df['Circonscription'].map(lambda s: unidecode(s))
df['Circonscription'] = df['Circonscription'].map(lambda s: s.strip())
print('Number of districts :', len(df))

# Sort by ID for 1-to-1 matching (faster at runtime)
all_map_districts = [f['properties']['NM_CEP'] for f in districts['features']]
df['ID'] = [all_map_districts.index(circo) for circo in df['Circonscription'].values]
df = df.sort_values(by='ID').reset_index(drop=True)

def get_map(demographics, variable):
    z = demographics[variable].values
    print(z)
    fig = go.Figure(go.Choroplethmap(
        geojson=districts,
        featureidkey='properties.ID',
        locations=[f['properties']['ID'] for f in districts['features']],
        z=z,
        hovertext=[f['properties']['NM_CEP'] for f in districts['features']],
        marker_opacity=0.5, marker_line_width=2))
    fig.update_geos(
        projection=dict(
            type="conic conformal",
            parallels=[50, 46]))
    fig.update_layout(
        map=dict(center=dict(lat=54, lon=-68.5), zoom=3.7),
        width=600, height=800)
    return fig


In [ ]:
# Assert that the dataframes are correctly ordered etc
print(len(df))

# Test Double inclusion
for i in range(125):
    assert df.loc[i, 'Circonscription'] in all_map_districts, f'{df.loc[i, "Circonscription"]} not in the map'
    assert all_map_districts[i] in df['Circonscription'].values, f'{districts["features"][i]["properties"]["NM_CEP"]} not in the dataframe'
    
# Test ID matching
for i in range(125):
    id, name = districts['features'][i]['properties']['ID'], districts['features'][i]['properties']['NM_CEP']
    assert name == df.loc[df['ID']==id, 'Circonscription'].values, f'{districts["features"][i]["properties"]["ID"]} not equal to {df.loc[i,"ID"]}'
    
# Test ordering by ID
for i in range(125):
    assert all_map_districts[i] == df.loc[i,'Circonscription'], f'{districts["features"][i]["properties"]["NM_CEP"]} not equal to {df.loc[i,"Circonscription"]}'

In [ ]:
fig = get_map(df, "Population totale selon les groupes d'age")
fig.show()